# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

# import warnings
# warnings.filterwarnings("ignore")

### Get data from `.csv` files and then preprocess data

In [2]:
def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    
    # Read English movie reviews
    df_eng = pd.read_csv("data/movie_reviews_eng_.csv")
    df_eng = df_eng.rename(columns={'Movie / TV Series': 'Title', 'Year': 'Year', 'Synopsis': 'Synopsis', 'Review': 'Review'})
    df_eng['Original Language'] = 'English'
    
    # Read French movie reviews
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={'Titre': 'Title', 'Année': 'Year', 'Synopsis': 'Synopsis', 'Critiques': 'Review'})
    df_fr['Original Language'] = 'French'
    
    # Read Spanish movie reviews
    df_sp = pd.read_csv("data/movie_reviews_sp_.csv")
    df_sp = df_sp.rename(columns={'Título': 'Title', 'Año': 'Year', 'Sinopsis': 'Synopsis', 'Críticas': 'Review'})
    df_sp['Original Language'] = 'Spanish'
    
    # Concatenate all dataframes into one resultant dataframe called "df"
    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    
    return df

df = preprocess_data()

In [3]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",French
7,The Nice Guys,2016,"In 1970s Los Angeles, a private eye (Ryan Gosl...","""The Nice Guys tries too hard to be funny, and...",English
20,Roma,2018,Cleo (Yalitza Aparicio) es una joven empleada ...,"""Roma es una película hermosa y conmovedora qu...",Spanish
11,Intouchables,2011,Ce film raconte l'histoire de l'amitié improba...,"""Intouchables est un film incroyablement touch...",French
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",French
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",Spanish
6,Scott Pilgrim vs. the World,2010,Scott Pilgrim (Michael Cera) must defeat his n...,"""It was difficult to sit through the whole thi...",English
14,Le Fabuleux Destin d'Amélie Poulain,2001,Cette comédie romantique raconte l'histoire d'...,"""Le Fabuleux Destin d'Amélie Poulain est un fi...",French
27,El Bar,2017,Un grupo de personas quedan atrapadas en un ba...,"""El Bar es una película ridícula y sin sentido...",Spanish
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",English


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [4]:
# Define the model names for the translation tasks
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

In [5]:
# Create the models and tokenizers
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

In [6]:
# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return decoded

In [7]:
# Filter reviews in French and translate them to English
df_fr = df[df["Original Language"] == "French"].copy()
fr_reviews = df_fr["Review"]
fr_reviews_en = fr_reviews.apply(lambda fr_reviews: translate(fr_reviews, fr_en_model,fr_en_tokenizer))

fr_synopsis = df_fr["Synopsis"]
fr_synopsis_en = fr_synopsis.apply(lambda fr_synopsis: translate(fr_synopsis, fr_en_model,fr_en_tokenizer))

In [8]:
# filter reviews in Spanish and translate to English
df_es = df[df["Original Language"] == "Spanish"]
es_reviews = df_es["Review"]
es_reviews_en = es_reviews.apply(lambda es_reviews: translate(es_reviews, es_en_model,es_en_tokenizer))

# filter synopsis in Spanish and translate to English
es_synopsis = df_es["Synopsis"]
es_synopsis_en = es_synopsis.apply(lambda es_synopsis: translate(es_synopsis, es_en_model,es_en_tokenizer))

In [15]:
es_reviews_en, fr_synopsis_en

(20    "Rome is a beautiful and moving film that pays...
 21    "The Paper House is an exciting and addictive ...
 22    "And your mom is also a movie that stays with ...
 23    "The Labyrinth of Fauno is a fascinating and e...
 24    "Amores dogs is an intense and moving film tha...
 25    "Red Eagle is a boring and uninteresting serie...
 26    "Toc Toc is a boring and unoriginal film that ...
 27    "The Bar is a ridiculous and meaningless film ...
 28    "Torrente is a vulgar and offensive film that ...
 29    "The Incident is a boring and frightless film ...
 Name: Review, dtype: object,
 10    This musical tells the story of a budding actr...
 11    This film tells the story of the unlikely frie...
 12    This romantic comedy tells the story of Amélie...
 13    This film tells the story of a music teacher w...
 14    This romantic comedy tells the story of Amélie...
 15    The film follows the story of a group of rich ...
 16    Two incompetent office workers find themselves...
 

In [17]:
# Filter out French reviews and synopses
df_filtered = df[df["Original Language"] != "French"]

# Update DataFrame with translated reviews and synopses for French
df_fr["Review"] = fr_reviews_en
df_fr["Synopsis"] = fr_synopsis_en

# Filter out Spanish reviews and synopses
df_filtered = df_filtered[df_filtered["Original Language"] != "Spanish"]

# Update DataFrame with translated reviews and synopses for Spanish
df_es["Review"] = es_reviews_en
df_es["Synopsis"] = es_synopsis_en

# Concatenate the modified DataFrames (English reviews and synopses) with the filtered DataFrame
df = pd.concat([df_filtered, df_fr, df_es], ignore_index=True)

C:\Users\oluse\AppData\Local\Temp\ipykernel_24932\3060449894.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_es["Review"] = es_reviews_en
C:\Users\oluse\AppData\Local\Temp\ipykernel_24932\3060449894.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_es["Synopsis"] = es_synopsis_en


In [18]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",French
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",Spanish
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",Spanish
23,El Laberinto del Fauno,2006,"During the Spanish postwar period, Ofelia (Iva...","""The Labyrinth of Fauno is a fascinating and e...",Spanish
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",French
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [24]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model= model_name)

In [25]:
# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    result = classifier(text)
    return result

In [26]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column
reviews = df["Review"]
results = reviews.apply(lambda reviews: analyze_sentiment(reviews,sentiment_classifier))
df["Sentiment"] = results.apply(lambda results: results[0]['label'])

In [31]:
results

0     [{'label': 'POSITIVE', 'score': 0.999888062477...
1     [{'label': 'POSITIVE', 'score': 0.999881744384...
2     [{'label': 'POSITIVE', 'score': 0.999884843826...
3     [{'label': 'POSITIVE', 'score': 0.999883770942...
4     [{'label': 'POSITIVE', 'score': 0.999885082244...
5     [{'label': 'NEGATIVE', 'score': 0.999675154685...
6     [{'label': 'NEGATIVE', 'score': 0.999802172183...
7     [{'label': 'NEGATIVE', 'score': 0.999802410602...
8     [{'label': 'NEGATIVE', 'score': 0.999798834323...
9     [{'label': 'NEGATIVE', 'score': 0.999816358089...
10    [{'label': 'POSITIVE', 'score': 0.999892830848...
11    [{'label': 'POSITIVE', 'score': 0.999881267547...
12    [{'label': 'POSITIVE', 'score': 0.999891877174...
13    [{'label': 'POSITIVE', 'score': 0.999893188476...
14    [{'label': 'POSITIVE', 'score': 0.999892711639...
15    [{'label': 'NEGATIVE', 'score': 0.999705612659...
16    [{'label': 'NEGATIVE', 'score': 0.999815285205...
17    [{'label': 'NEGATIVE', 'score': 0.99979847

In [27]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",French,POSITIVE
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",English,POSITIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",English,NEGATIVE
5,Blade Runner 2049,2017,"Officer K (Ryan Gosling), a new blade runner f...","""Boring and too long. Nothing like the origina...",English,NEGATIVE
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",French,NEGATIVE
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",French,POSITIVE
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",English,POSITIVE
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",English,POSITIVE
22,Y tu mamá también,2001,Two teenage friends (Gael García Bernal and Di...,"""And your mom is also a movie that stays with ...",Spanish,POSITIVE
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",French,POSITIVE


In [30]:
# export the results to a .csv file
df.to_csv("data/output/reviews_with_sentiment.csv", index=False)